In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [3]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

### Our mean is not near 0 and our std deviation is not near 1 so let's do something about that.

In [6]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [8]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

## We want our data to have a mean near 0 and a std deviation of 1

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [11]:
y_train.max()

tensor(9)

We have 10 possible options as outputs -- 0 to 9.

## Foundations version

### Basic architecture

In [12]:
# num hidden
nh = 50

Note that by calling torch.randn we get numbers that have a mean of 0 and std-dev of 1.
From docs: Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1 (also called the standard normal distribution).

In [13]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [14]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

## We also want our initialized weights to have a mean near 0 and a std deviation of 1

In [15]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [16]:
def lin(x, w, b): return x@w + b

In [17]:
t = lin(x_valid, w1, b1)

In [18]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0058), tensor(0.9597))

In [19]:
def relu(x): return x.clamp_min(0.)

In [22]:
# this is actually what our first layer looks like rather than just the linear function.
# we have a linear and a relu in our first layer and we want the result of this to have a mean of 0 and std-dev of 1
t = relu(lin(x_valid, w1, b1))

In [20]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.0058), tensor(0.9597))

In [21]:
test_near_zero(t.mean())

AssertionError: Near zero: 0.0057941642589867115

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [78]:
# kaiming init / he init for relu
# big change is we multiply by 2
w1 = torch.randn(m,nh)*math.sqrt(2/m)

Note that we use m (the number of inputs). This is for the forward pass.
On backwards pass, would use nh.
The default in pytorch on init is forward (fain_in) but they have the inputs and outputs reversed, so have to specify "fan_out" as option as seen below.

In [79]:
w1.mean(),w1.std()

(tensor(0.0001), tensor(0.0509))

In [80]:
test_near_zero(w1.mean())

In [85]:
# this gives us a much better variance but mean is still not that close to zero
# because we got rid of all the negatives with our relu so it is around 0.5 for the mean rather than 0
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5525), tensor(0.8020))

In [86]:
test_near_zero(t.std() - 1)

AssertionError: Near zero: -0.19803869724273682

In [87]:

test_near_zero(t.mean())

AssertionError: Near zero: 0.5525319576263428

In [88]:
# could try to a relu function that is min(0, x) - 0.5 to stay with 0 mean
# jeremy is experimenting with this

In [90]:
#export
from torch.nn import init

In [91]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [98]:
init.kaiming_normal_??

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}$$

In [94]:
w1.mean(),w1.std()

(tensor(-0.0005), tensor(0.0503))

In [95]:
t.mean(),t.std()

(tensor(0.5811), tensor(0.8535))

Again, was pretty good at getting std-dev close to 1. mean is at 0.5 rather than 0.

In [96]:
w1.shape

torch.Size([784, 50])

In [103]:
import torch.nn

In [104]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [105]:
torch.nn.Linear.forward??

In [106]:
torch.nn.functional.linear??

From looking at the functions we see that F.linear is doing a matrix multiplicaton on the TRANSPOSED weights.
That is why we have to specify fan_out on our initialization even though we're doing a forward pass.

In [107]:
torch.nn.Conv2d??

In [108]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [109]:
# what if...?
# playing at trying to get the mean to 0. seems to also help with std-dev
def relu(x): return x.clamp_min(0.) - 0.5

In [111]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0116), tensor(0.7880))

In [112]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [121]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [122]:
%timeit -n 10 _=model(x_valid)

4.68 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [123]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

In [120]:
# output of our model should be number of entries in our x_valid set by 1 for 1 prediction for each
x_valid.shape

torch.Size([10000, 784])

### Loss function: MSE

Using MSE even though should use cross-entropy for categorization. We are only getting one final activation when we really want 10 (1 for each possible digit).

In [124]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

### Should always use an argument with .squeeze(). it gets rid of all dimensions that have a 1. This means that if you have a batch size of 1 it will break your code.  By saying -1, it gets rid of the last dimension which is what we want here. Alternatively, could do squeeze(1) since we want to get rid of the 1st dimension (0, indexed of course).

In [125]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

For mse, will need floats so convert to floats.

In [126]:
y_train[0]

tensor(5)

In [127]:
y_train,y_valid = y_train.float(),y_valid.float()

In [128]:
y_train[0]

tensor(5.)

In [129]:
preds = model(x_train)

In [130]:
preds.shape

torch.Size([50000, 1])

In [131]:
mse(preds, y_train)

tensor(31.4793)

### Gradients and backward pass

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [149]:
# need derivative of: (output.squeeze(-1) - targ).pow(2).mean()
def my_mse_grad(inp, targ):
    inp.g = 2. * (inp.squeeze(-1) - targ).unsqueeze(-1)/inp.shape[0]

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [134]:
def my_relu_grad(inp, out):
#     derivative of relu is just 0 if value is less than 0 and 1 if value is greater than 0
    derivative = (inp > 0).float()
    inp.g = derivative * out.g

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [150]:
def my_lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [146]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [151]:
def my_forward_and_backward(inp, targ):
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    loss = mse(out, targ)
    
#     backward
    my_mse_grad(out, targ)
    my_lin_grad(l2, out, w2, b2)
    my_relu_grad(l1, l2)
    my_lin_grad(inp, l1, w1, b1)

In [152]:
my_forward_and_backward(x_train, y_train)

In [153]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [154]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [155]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [156]:
loss = forward(xt2, y_train)

In [157]:
loss.backward()

In [158]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [159]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [160]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [161]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [162]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [163]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [164]:
%time loss = model(x_train, y_train)

CPU times: user 136 ms, sys: 0 ns, total: 136 ms
Wall time: 36.7 ms


In [165]:
%time model.backward()

CPU times: user 4.02 s, sys: 3.09 s, total: 7.11 s
Wall time: 6.18 s


In [166]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [167]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [168]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [169]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [172]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze(-1)-targ).unsqueeze(-1) / targ.shape[0]

In [173]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [174]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [175]:
%time loss = model(x_train, y_train)

CPU times: user 140 ms, sys: 0 ns, total: 140 ms
Wall time: 35.8 ms


In [176]:
%time model.backward()

CPU times: user 840 ms, sys: 72 ms, total: 912 ms
Wall time: 262 ms


In [177]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [178]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [179]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [180]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 4 ms, total: 124 ms
Wall time: 33.1 ms


In [181]:
%time model.backward()

CPU times: user 552 ms, sys: 92 ms, total: 644 ms
Wall time: 163 ms


In [182]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [183]:
#export
from torch import nn

In [188]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(-1), targ)

In [189]:
model = Model(m, nh, 1)

In [190]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 34.1 ms


In [187]:
%time loss.backward()

CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 49.9 ms


## Export

In [ ]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
